<a href="https://colab.research.google.com/github/Ign4cho/practicas/blob/main/transfomacion_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime
import gspread as gs
import numpy as np
from google.colab import auth
from google.auth import default

# Revisión CSV
falta implementar

- tomar fecha de inicio y fecha de fin
- que hacer con las que tienen salida y no entrada

me gustaría implementar
- hojas auxiliares con campos calculados y tablas auxiliares
  - ver que hojas se podrían calcular
- en otra tabla se pueden mapear
    - "ORIGEN" y "ORIGENDOCID"
    - ESTADO y IDESTADO
    - DOCUMENTOSITUCION y DOCUMENTOSITUACIONID
    - USUARIOREGISTRO y USUARIOREGISTROID creo que son lo mismo
    - USUARIOREGISTROID, USUARIOREGISTROCUENTA y USUARIOREGISTRONOMBRE
    - DOCUMENTOESPECIALIDADID y DOCUMENTOESPECIALIDAD
    - DOCUMENTOOCUPACIONID y DOCUMENTOOCUPACION

arreglar
-

# Carga de datos

In [ ]:
#cargamos las credenciales, este código es del ejemplo de colab
auth.authenticate_user()
credenciales, _ = default()
gc = gs.authorize(credenciales)

#montamos el drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# importamos desde drive
file_path = '/content/drive/MyDrive/Práctica Profesional/VW_SGR_REVISIONDOCUMENTOS_202409122100.csv'
df = pd.read_csv(file_path)
# almacenamos la fecha del .csv
fecha = file_path.split("_")[-1].split(".")[0]

In [ ]:
# donde se va a guardar el archivo de salida
#el id carpeta destino es necesario para crear el archivo en caso de ser necesario

file_path_destino = "revsion_csv"
id_carpeta_destino = "1oMTHv24S5rNlxhwignHef2_9HlrmpTwh"


# Fecha de inicio y fin de la revisión

fecha_inicial = datetime(2024, 8, 1)
fecha_final = datetime(2024, 8, 10)


# Transformación

In [ ]:
#eliminamos las columnas no relevantes para el análisis

columnas_no_tenidas_en_cuenta = [
 'FECHA_REGISTRO',
 'DOCO_EMPR_RUT',
 'DOCO_CONT_ID',
 'DOCUMENTOSITUCION',
 'DOCUMENTOESPECIALIDAD',
 'FECHA_INICIO_DOCUMENTO',
 'FECHA_FIN_DOCUMENTO',
 'DOCUMENTOVIGENCIA',
 'DOCO_TIPO_VALIDACION',
 'DOCO_CONT_ID'
]

df.drop(columnas_no_tenidas_en_cuenta, axis=1, inplace=True)


In [ ]:
df.shape

(58491, 19)

In [ ]:
# Creamos la columna tipo, en base a la columna estado.
# Si el estado es "Enviado", el tipo es de "Entrada"
# Si el estado es "Rechazado" o "Validado", el tipo es de "Salida"

df["TIPO"] = df["ESTADO"].apply(lambda x: "Entrada" if x == "Enviado" else "Salida")

In [ ]:
# Convertimos la columna FECHA_REGISTRO_HH, en formato aaaa-mm-dd hh:mm:ss.mmm a datetime
df["FECHA_REGISTRO_HH"] = pd.to_datetime(df["FECHA_REGISTRO_HH"], format="%Y-%m-%d %H:%M:%S.%f")


In [ ]:
#creamos dos dataframes, uno con los documentos de entrada y otro con los de salida
df_entrada = df[df["TIPO"] == "Entrada"]
df_salida = df[df["TIPO"] == "Salida"]

In [ ]:
columnas_eliminadas_salida = ['ORIGENDOCID', 'ORIGEN', 'DOCO_TIDO_ID',
       'ID_ESTADO', 'DOCUMENTOSITUACIONID', 'USUARIOREGISTRO',
       'USUARIOREGISTROCUENTA', 'USUARIOREGISTRONOMBRE',
       'FECHA_INGRESO_DOCUMENTO', 'MOTIVO_RECHAZO', 'DOCUMENTOESPECIALIDADID',
       'DOCUMENTOOCUPACIONID', 'DOCUMENTOOCUPACION',
       'DOCUMENTOCERTOCUPACIONID', 'DOCUMENTOCERTOCUPACION', 'TIPO']


In [ ]:
# del dataframe de salidas, nos interesa conservar las columnas
# FECHA_REGISTRO_HH, ESTADO, USUARIOREGISTROID, pero renombramos a
# FECHA_SALIDA, ESTADO_SALIDA, USUARIOID_SALIDA  para luego poder hacer el merge
df_salida = df_salida.rename(columns={"FECHA_REGISTRO_HH": "FECHA_SALIDA", "ESTADO": "ESTADO_SALIDA", "USUARIOREGISTROID": "USUARIOID_SALIDA"})
df_salida.drop(
       columnas_eliminadas_salida
       , axis=1, inplace=True)


In [ ]:
# hacemos el merge, en base al doco_id, lo hacemos de tipo left para conservar los documentos de entrada
df_merged = pd.merge(df_entrada, df_salida, on="DOCO_ID", how="left")


In [ ]:
#Agregamos una columna con la diferencia de tiempo entre entrada y salida de los documentos en horas
# en caso de no haber fecha salida, la columna se rrrellena con NaN
# en caso de no haber fecha de registro, la columna se rellena con NaN

df_merged["DIFERENCIA_HORAS"] = df_merged.apply(
    lambda row: (row["FECHA_SALIDA"] - row["FECHA_REGISTRO_HH"]).total_seconds() / 3600
    if pd.notnull(row["FECHA_SALIDA"]) and pd.notnull(row["FECHA_REGISTRO_HH"]) else np.nan,
    axis=1
)

In [ ]:
# Definimos el umbral de 48 horas
umbral = 48

# Agregamos un campo de desviación respecto al umbral
df_merged["DESVIACION"] = df_merged["DIFERENCIA_HORAS"] - umbral

# Determinamos el estado, teniendo en cuenta la desviación y los documentos
# sin salida

df_merged['estado'] = df_merged.apply(
    lambda row: 'Sin gestionar' if pd.isna(row['FECHA_SALIDA'])
    else 'Sin entrada' if pd.isna(row['FECHA_REGISTRO_HH'])
    else 'Fuera de plazo' if row['DESVIACION'] > 0
    else 'En plazo' if row['DESVIACION'] < 0
    else 'Igual',
    axis=1
)


In [ ]:
# Obtener el mes actual y los tres meses anteriores
hoy = datetime.now()
mes_actual = hoy.month
año_actual = hoy.year

# Crear una lista de los tres meses anteriores y el mes actual
meses_validos = [(mes_actual - i - 1) % 12 + 1 for i in range(4)]
año_valido = año_actual if mes_actual in meses_validos else año_actual - 1

# Función para obtener la etiqueta del mes
def etiqueta_mes(fecha):
    mes = fecha.month
    año = fecha.year

    if mes == mes_actual and año == año_actual:
        return f"{hoy.strftime('%B')} {año_actual}"

    if mes in meses_validos:
        return f"{fecha.strftime('%B')} {año}"

    return 'Fuera de rango'

# Aplicar la función al DataFrame
df_merged['etiqueta_mes'] = df_merged['FECHA_REGISTRO_HH'].apply(etiqueta_mes)

In [ ]:
# filtramos el dataframe en por fecha de salida

df_merged = df_merged[(df_merged['FECHA_SALIDA'] >= fecha_inicial) & (df_merged['FECHA_SALIDA'] <= fecha_final)]

# Preaparamos una segunda hoja para el análisis de datos de los docs procesados por IA


In [ ]:
# creamos un segundo dataframe con los documentos gestionados con IA
# el id de usurario de la IA es 6, el USUARIOREGISTROCUENTA SAT_INTEGRACION
df_SAT = df_merged[df_merged['USUARIOID_SALIDA'] == 6]

# convertimos los timestamps a string, teniendo en cuenta que pueden haber NaTs

df_SAT = df_SAT.map(
    lambda x: x.strftime("%Y-%m-%d %H:%M:%S")
    if isinstance(x, (pd.Timestamp, datetime)) and not pd.isnull(x) else x
)

# reemplazamos NaNs con espccios vacíos
df_SAT= df_SAT.map(lambda x: '' if pd.isnull(x) else x)


In [ ]:
#exportamos los datos como segunda página del Sheet
try:
  sh = gc.open(file_path_destino)
except gs.SpreadsheetNotFound:
  sh = gc.create(file_path_destino, folder_id=id_carpeta_destino)

nombre_hoja = 'SAT_integracion'
try:
  worksheet = sh.worksheet(nombre_hoja)
except gs.WorksheetNotFound:
  sh.add_worksheet(nombre_hoja, rows=100000, cols=80)
  worksheet = sh.worksheet(nombre_hoja)
worksheet.update([df_SAT.columns.values.tolist()] + df_SAT.values.tolist())

{'spreadsheetId': '1K6qyS0JNMn2MxryDT9orMpEIbRFufDQETVSNHDn9Q7E',
 'updatedRange': 'SAT_integracion!A1:AA3380',
 'updatedRows': 3380,
 'updatedColumns': 27,
 'updatedCells': 91260}

# Exportación a Google drive

In [ ]:
#Preparamos la exprotación a sheets
#  es necesario convertir los timestamps a string, en formato yyyy-mm-dd hh:mm:ss
# convertimos los timestamps a string, teniendo en cuenta que pueden haber NaTs

df_merged = df_merged.map(
    lambda x: x.strftime("%Y-%m-%d %H:%M:%S")
    if isinstance(x, (pd.Timestamp, datetime)) and not pd.isnull(x) else x
)

# reemplazamos NaNs con espccios vacíos
df_merged= df_merged.map(lambda x: '' if pd.isnull(x) else x)



In [ ]:
# exportamos el dataframe a sheets
try:
  sh = gc.open(file_path_destino)
except gs.SpreadsheetNotFound:
  sh = gc.create(file_path_destino, folder_id=id_carpeta_destino)

worksheet = sh.sheet1
worksheet.update([df_merged.columns.values.tolist()] + df_merged.values.tolist())


{'spreadsheetId': '1K6qyS0JNMn2MxryDT9orMpEIbRFufDQETVSNHDn9Q7E',
 'updatedRange': "'Hoja 1'!A1:AA29634",
 'updatedRows': 29634,
 'updatedColumns': 27,
 'updatedCells': 800118}